This code is leveraged by Chatgpt and the prompt is given below
1.I only want trending news and nothing else which can help me analyse the trend by which news is the most trending recently or which one is the most recent like that
2.I have multiple json files which contains news can you make the same code for them files
3.The title I am getting is nan so I have to get a proper headline detail can you help me with that
This code relies on frequency analysis and sorting to determine trending news.




In [13]:
import os  # For working with files and directories
import json  # For reading JSON files
import pandas as pd  # For handling tabular data
from collections import Counter  # For counting occurrences of words

# 📂 Define the folder path where JSON files are stored
folder_path = r"C:\Users\User\Desktop\second year\Big Data Capstone - Bhavik\NewsData Download Database"

# 📋 Initialize an empty list to store news articles from all JSON files
news_list = []

# 🔄 Loop through all files in the folder
for file in os.listdir(folder_path):
    if file.endswith(".json"):  # ✅ Process only JSON files
        file_path = os.path.join(folder_path, file)  # 📌 Get the full file path
        
        with open(file_path, "r", encoding="utf-8") as f:  # 📖 Open and read the JSON file
            data = json.load(f)  # 🔄 Load the JSON content
            
            # ✅ Check if the JSON file contains a list of news articles
            if isinstance(data, list):
                news_list.extend(data)  # 📥 Append all articles to the main list
            
            # ✅ If the JSON file contains a single news article as a dictionary
            elif isinstance(data, dict):
                news_list.append(data)  # 📥 Convert it to a list and append

# 🏗 Convert the list of news articles into a Pandas DataFrame
df = pd.DataFrame(news_list)

# 🛠 Debugging: Print column names to check what exists in the dataset
print("\n📌 Available columns in the dataset:", df.columns.tolist())

# 🔍 Define possible names for the publication date column (varies across datasets)
possible_date_columns = ["published_date", "timestamp", "date", "pub_date"]

# 🔄 Loop through the list of possible date column names
for col in possible_date_columns:
    if col in df.columns:  # ✅ If the column exists in the dataset
        df.rename(columns={col: "published_date"}, inplace=True)  # 🔄 Standardize its name
        break  # ⏹ Stop checking once a match is found

# ⚠️ If no valid date column is found, print an error and exit the script
else:
    print("❌ No valid 'published_date' column found! Check your JSON structure.")
    print(df.head())  # 🔎 Print the first few rows for debugging
    exit()  # 🚨 Stop execution since the date column is necessary

# 🕒 Convert 'published_date' column to datetime format (handling errors gracefully)
df["published_date"] = pd.to_datetime(df["published_date"], errors="coerce")

# 🗑 Remove rows where 'published_date' could not be converted (invalid/missing dates)
df = df.dropna(subset=["published_date"])

# 🔽 Sort articles by 'published_date' in descending order (most recent first)
df = df.sort_values(by="published_date", ascending=False)

# 🔍 Check if the dataset contains 'title' column; otherwise, use 'headline'
if "title" in df.columns and df["title"].notna().sum() > 0:
    df["final_title"] = df["title"]  # ✅ Use 'title' if available
elif "headline" in df.columns:
    df["final_title"] = df["headline"]  # ✅ Use 'headline' if 'title' is missing
else:
    print("❌ No 'title' or 'headline' column found in the dataset.")  # 🚨 Error message
    exit()  # ⏹ Stop execution since a title is required

# 🔡 Convert all titles to lowercase and split into individual words (for keyword analysis)
df["title_keywords"] = df["final_title"].str.lower().str.split()

# 🔢 Flatten the list of words from all news titles and count word occurrences
all_keywords = [word for words in df["title_keywords"].dropna() for word in words]
trending_keywords = Counter(all_keywords).most_common(10)  # 🔥 Find the top 10 most common words

# 📊 Print the most mentioned words (trending keywords)
print("\n🔥 Most Mentioned Keywords in Headlines:")
print(trending_keywords)

# 📰 Print the top 10 most recent trending news articles
print("\n🔹 Top Trending News Articles (Most Recent First):")
print(df[["final_title", "published_date"]].head(10))



📌 Available columns in the dataset: ['_id', 'headline', 'summary', 'link', 'timestamp', 'title', 'Summary', 'category']

🔥 Most Mentioned Keywords in Headlines:
[('to', 777), ('in', 772), ('of', 485), ('the', 416), ('and', 310), ('on', 310), ('a', 310), ('as', 283), ('for', 280), ('says', 268)]

🔹 Top Trending News Articles (Most Recent First):
                                            final_title  \
6416  Pennsylvania's governor says the Trump adminis...   
6415  US says OKX crypto exchange operator enters $5...   
6414  ‘Devastating’: Mass. school district mourning ...   
6413  Intel says new ASML machines are in production...   
6412  Judge upholds White House ban on AP amid Gulf ...   
6411  Who is Dan Bongino, the new deputy director of...   
928   Leeds stun Sheff Utd with comeback to go five ...   
927   Ex-surgeon admits 'despicable acts' in French ...   
926   Federal worker unions sue Musk over demand the...   
925   Peace must not mean surrendering Ukraine, Macr...   

  